# <center>线性判别分析</center>

## 思想
类间大+类内小

映射矩阵$w \in R^{n \times k}$
将$x \in R^n$ 通过$y \in R^k = w^T x$ 映射到k维
$k=num_{labels} - 1$

数据集
$$D={x^{(1)},x^{(2)},...,x^{(m)}}$$
将D按照类别标签划分为两个子集$D_1$,$D_2$

两个子集的中心为
$$\mu_1=\frac{1}{N_1} \sum_{x^{(i)} \in D_1} x^{(i)}$$
$$\mu_2=\frac{1}{N_2} \sum_{x^{(i)} \in D_2} x^{(i)}$$

两个子集投影之后的中心为
$$w^T \mu_1$$
$$w^T \mu_2$$


两个子集投影之后的方差为
$$\sigma_1^2
=\frac{1}{N_1} \sum_{x^{(i)} \in D_1} (w^T x^{(i)} - w^T \mu_1)^2
=\frac{1}{N_1} \sum_{x^{(i)} \in D_1} w^T (x^{(i)} - \mu_1) (x^{(i)} - \mu_1)^T w
$$
$$\sigma_2^2
=\frac{1}{N_2} \sum_{x^{(i)} \in D_2} (w^T x^{(i)} - w^T \mu_2)^2
=\frac{1}{N_2} \sum_{x^{(i)} \in D_2} w^T (x^{(i)} - \mu_2) (x^{(i)} - \mu_2)^T w
$$

令
$$S_1 = \frac{1}{N_1} \sum_{x^{(i)} \in D_1} (x^{(i)} - \mu_1) (x^{(i)} - \mu_1)^T$$
$$S_2 = \frac{1}{N_2} \sum_{x^{(i)} \in D_2} (x^{(i)} - \mu_2) (x^{(i)} - \mu_2)^T$$

则
$$\sigma_1^2 = w^T S_1 w$$
$$\sigma_2^2 = w^T S_2 w$$

定义优化目标函数
$$J(w)
=\frac{|w^T \mu_1 - w^T \mu_2|^2}{\sigma_1^2 + \sigma_2^2}
=\frac{w^T (\mu_1 - \mu_2)(\mu_1 - \mu_2)^T w}{w^T (S_1 + S_2) w}
$$

令
$$S_B = (\mu_1 - \mu_2)(\mu_1 - \mu_2)^T$$
$$S_W = S_1 + S_2$$

则
$$J(w) = \frac{w^T S_B w}{w^T S_W w}$$

$$w = \underset{w}{\operatorname{argmax}} J(w)$$

In [2]:
from collections import namedtuple
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets


def shuffle_data(X, y, seed=None):
    if seed:
        np.random.seed(seed)
    idx = np.arange(len(X))
    np.random.shuffle(idx)
    return X[idx], y[idx]


def standardize(X):
    column_mean = X.mean(axis=0)
    column_std = X.std(axis=0)

    X_std = np.zeros(X.shape)
    for col in range(X.shape[1]):
        if column_std[col] != 0:
            X_std[:, col] = (X[:, col] - column_mean[col]) / column_std[col]
    return X_std


def train_test_split(X, y, test_size=0.2, shuffle=True, seed=None):
    if shuffle:
        X, y = shuffle_data(X, y, seed)
    n_train_samples = int(len(X) * (1 - test_size))
    X_train, X_test = X[:n_train_samples], X[n_train_samples:]
    y_train, y_test = y[:n_train_samples], y[n_train_samples:]
    return X_train, X_test, y_train, y_test


def accuracy(y, y_pred):
    return np.sum(y == y_pred) / len(y)


def covariance_matrix(X, Y=np.empty((0, 0))):
    if not Y.any():
        Y = X
    n_samples = len(X)
    cov_mat = (1 / (n_samples - 1)) * (X - X.mean(axis=0)).T.dot(Y - Y.mean(axis=0))
    return np.array(cov_mat, dtype=float)


class LinearDiscriminantAnalysis():
    def __init__(self):
        self.w = None

    def fit(self, X, y):
        X1 = X[y == 0]
        X2 = X[y == 1]

        S1 = covariance_matrix(X1)
        S2 = covariance_matrix(X2)
        Sw = S1 + S2

        mu1 = X1.mean(axis=0)
        mu2 = X2.mean(axis=0)

        self.w = np.linalg.pinv(Sw).dot(mu1 - mu2)

    def predict(self, X):
        y_pred = []
        for sample in X:
            y_pred.append(1 * (np.dot(sample, self.w) < 0)) #阈值为0
        return y_pred


data = datasets.load_iris()
X = data.data
y = data.target
X = X[y < 2]
y = y[y < 2]

X_train, X_test, y_train, y_test = train_test_split(X, y)

model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

print(f'Accuracy:{accuracy(y_test, model.predict(X_test))}')
print((X_test @ model.w).reshape(4,-1))
print(np.array(model.predict(X_test)).reshape(4,-1))

Accuracy:1.0
[[ 18.00299579 -39.62031461  18.67549754 -28.7878528   14.38963711]
 [-34.46165684 -26.34604273  11.6479477   10.24517246  14.33186538]
 [-50.51511773 -41.46884885  16.7029347   12.16309396 -31.52088562]
 [ 15.00436714 -30.5034764  -39.90200985  15.04499711 -33.99258104]]
[[0 1 0 1 0]
 [1 1 0 0 0]
 [1 1 0 0 1]
 [0 1 1 0 1]]
1
